In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda, Flatten
from keras.layers import Convolution1D, Dense, Dropout, MaxPooling1D, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from keras import callbacks
from keras.callbacks import CSVLogger
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler 
import pickle
from sklearn import metrics
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas  as pd
import numpy as np
import os
import time
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
        f1_score, classification_report, 
        confusion_matrix, roc_curve, 
        roc_auc_score, accuracy_score,
        log_loss)
from sklearn import __version__ as sklearn_version
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import NearMiss
from datetime import datetime
np.random.seed(0)

In [ ]:
dir2='/content/drive/MyDrive/botnet_dataset/device6/'
device2_df=pd.DataFrame()
i=0
for file in os.listdir(dir2+'gafgyt/'):
  df_file = pd.read_csv(os.path.join(dir2+'gafgyt/', file), encoding = 'ISO-8859-1',low_memory=False)
  df_file["label"]=i
  i=i+1
  device2_df=pd.concat([device2_df, df_file])
for file in os.listdir(dir2+'mirai/'):
  df_file = pd.read_csv(os.path.join(dir2+'mirai/', file), encoding = 'ISO-8859-1',low_memory=False)
  df_file["label"]=i
  i=i+1
  device2_df=pd.concat([device2_df, df_file])
df_file=pd.read_csv('/content/drive/MyDrive/botnet_dataset/device6/benign.csv')
df_file['label']=i
device2_df=pd.concat([device2_df, df_file])

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(device2_df, test_size=0.2)


In [ ]:
X_train=train.drop(['label'],axis=1)
y_train=train['label']


In [ ]:
from sklearn.tree import DecisionTreeClassifier
model_lr=DecisionTreeClassifier()
rfe = RFE(model_lr, n_features_to_select=50,step=4)
rfe.fit(X_train,y_train)

RFE(estimator=DecisionTreeClassifier(), n_features_to_select=50, step=4)

In [ ]:
from operator import itemgetter
features = X_train.columns.to_list()
i=0
feat_arr_rem=[]
for x, y in (sorted(zip(rfe.ranking_ , features), key=itemgetter(0))):
    if(x!=1):
      i=i+1
      feat_arr_rem.append(y)

In [ ]:
mod_df=device2_df.copy()
for str in feat_arr_rem:
  if(str=='label'):
    continue
  mod_df.drop([str],axis=1, inplace=True)
mod_df.head()

,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,MI_dir_L0.1_mean,MI_dir_L0.1_variance,MI_dir_L0.01_weight,MI_dir_L0.01_mean,MI_dir_L0.01_variance,H_L5_weight,H_L5_mean,...,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_pcc,label
0,98.000000,0.000000e+00,1.000000,98.000000,0.000000e+00,1.000000,98.000000,0.000000,1.000000,98.000000,...,0.000000e+00,0.0,0.0,1.000000,98.0,0.000000,98.000000,0.000000e+00,0.0,0
1,98.000000,3.637979e-12,1.931640,98.000000,1.818989e-12,1.992944,98.000000,0.000000,1.029000,98.000000,...,1.818989e-12,0.0,0.0,1.992944,98.0,0.000001,138.592929,1.818989e-12,0.0,0
2,84.051188,2.517926e+02,2.904273,86.981750,2.311822e+02,2.990102,87.298025,227.930928,1.504156,76.725612,...,0.000000e+00,0.0,0.0,1.000000,66.0,0.000000,114.856432,0.000000e+00,0.0,0
3,80.987267,1.964467e+02,3.902546,83.655268,2.040614e+02,3.989925,83.965124,204.017078,2.460087,75.617679,...,0.000000e+00,0.0,0.0,1.000000,74.0,0.000000,74.000000,0.000000e+00,0.0,0
4,79.354915,1.592943e+02,4.902545,81.685828,1.775746e+02,4.989924,81.968075,179.043909,3.460055,75.150149,...,0.000000e+00,0.0,0.0,1.000000,74.0,0.000000,74.000000,0.000000e+00,0.0,0


In [ ]:
mod_df.to_csv('/content/drive/MyDrive/botnet_dataset/device6/dataset_final.csv')

In [ ]:
temp_df=pd.read_csv('/content/drive/MyDrive/botnet_dataset/device6/dataset_final.csv')

In [ ]:
x=temp_df.drop(['label'],axis=1)
y=temp_df['label']
normalized_df=(x-x.mean())/x.std()
y.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(normalized_df, y, test_size=0.2, random_state=0)
y_train = np.array(y_train)

y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1]))
print('xtrain={}, ytrain={}, xtest={}, ytest={}'.format(X_train.shape,y_train.shape,X_test.shape,y_test.shape))

xtrain=(669512, 51), ytrain=(669512,), xtest=(167379, 51), ytest=(167379,)


In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()
start=time.time()
model.fit(X_train, y_train)
end=time.time()
tt=end-start

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_train.shape

(669512,)

In [ ]:
from sklearn import metrics
from sklearn.metrics import f1_score
print('XGBoost Classifier')
print(f"Training time: {tt}s")
print('Accuracy = ', metrics.accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix =\n", metrics.confusion_matrix(y_test, y_pred, labels=None, 
                                              sample_weight=None))
print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
print("Classification Report =\n", metrics.classification_report(y_test, y_pred, 
                                                                 labels=None, 
                                                                 target_names=None, 
                                                                 sample_weight=None, 
                                                                 digits=2, 
                                                                 output_dict=False))

print("F1 Score = ",f1_score(y_test, y_pred, average='macro'))

XGBoost Classifier
Training time: 1830.176795721054s
Accuracy =  99.9952204278912
Confusion Matrix =
 [[11637     0     0     0     0     0     0     0     0     0     0]
 [    1  5704     0     0     0     0     0     0     0     0     0]
 [    0     1  5740     1     0     0     0     0     0     0     0]
 [    0     0     1 17777     0     0     0     0     0     0     0]
 [    0     0     0     0 20911     0     0     0     0     0     1]
 [    0     0     0     1     0 11454     0     0     0     0     0]
 [    0     0     0     0     0     0 19458     0     0     0     0]
 [    0     0     0     0     0     0     0 12487     0     0     0]
 [    0     0     0     1     0     0     0     0 31877     0     0]
 [    0     0     0     0     0     0     0     0     0 10798     0]
 [    0     1     0     0     0     0     0     0     0     0 19528]]
Recall = 0.999952204278912
Classification Report =
               precision    recall  f1-score   support

           0       1.00      1.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def rf_classifier(data, device_name, scaling=False,):
    X = data.drop(['label'], axis=1)
    y = data['label']
    print(f'Original Shape:{X.shape}, {y.shape}')
    if scaling == False:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        model_name = f'{device_name}_without_scaling_unbalanced_model.pkl'
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        model_name = f'{device_name}_with_scaling_unbalanced_model.pkl'
    
    clf = RandomForestClassifier()
    start=time.time()
    model_res = clf.fit(X_train, y_train)
    end=time.time()
    tt=end-start
    y_pred = model_res.predict(X_test)
    y_pred_prob = model_res.predict_proba(X_test)
    lr_probs = y_pred_prob[:,1]
    ac = accuracy_score(y_test, y_pred)
    importances = pd.DataFrame({'feature':X.columns,'importance':np.round(clf.feature_importances_,3)})
    importances = importances.sort_values('importance',ascending=False).set_index('feature')

    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    print("Random Forest Classifier")
    print(f"Training time: {tt}s")
    print('Accuracy=%.3f' % (ac))
    print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
    print('f1-score=%.3f' % (f1))
    print('Confusion Matrix = \n', cm)
    print('Classification Report = \n',classification_report(y_test, y_pred))

In [ ]:
rf_classifier(temp_df,'device4',True)

Original Shape:(836891, 51), (836891,)
Random Forest Classifier
Training time: 201.2093689441681s
Accuracy=0.898
Recall = 0.898099319706215
f1-score=0.866
Confusion Matrix = 
 [[17357     0     1     2     0     0     0     0     0     0     0]
 [    1  8697     1     1     0     0     0     0     0     0     0]
 [    0     0  8505     0     1     0     0     0     0     0     0]
 [    0     0     0  1146 25574     0     0     0     0     0     0]
 [    0     0     0     0 31132     0     0     0     0     0     0]
 [    0     0     0     0     0 17373     0     0     0     0     0]
 [    0     0     0     0     0     0 29216     0     0     0     0]
 [    0     0     0     0     0     1     0 18589     0     0     1]
 [    0     0     0     0     0     0     0     1 47632     0     0]
 [    0     0     0     0     0     0     0     0     0 16075     0]
 [    0     0     0     0     0     0     0     0     0     0 29762]]
Classification Report = 
               precision    recall  f1-

In [ ]:
from sklearn import metrics
from sklearn.metrics import f1_score
def knn_classifier_unbalanced(data, scaling=False):
    X = data.drop(['label'], axis=1)
    y = data['label']
    print(f'Original Shape:{X.shape}, {y.shape}')
    test_scores = []
    train_scores = []
    error = []
    if scaling == False:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
    knn = KNeighborsClassifier(3)
    start=time.time()
    knn.fit(X_train,y_train)
    end=time.time()
    tt=end-start
    y_pred = knn.predict(X_test)
    error.append(np.mean(y_pred != y_test))
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))
    print("KNN")
    print(f"Training time: {tt}s")
    print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
    print('Classification Report = \n',classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(f'Accuracy on Training set:{train_scores}')
    print(f'Accuracy on Test set:{test_scores}')

In [23]:
knn_classifier_unbalanced(temp_df,True)

Original Shape:(836891, 51), (836891,)
KNN
Training time: 0.21754121780395508s
Recall = 0.9114104545382128
Classification Report = 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     17360
           1       1.00      1.00      1.00      8700
           2       1.00      1.00      1.00      8506
           3       0.59      0.58      0.59     26720
           4       0.64      0.65      0.64     31132
           5       1.00      1.00      1.00     17373
           6       1.00      1.00      1.00     29216
           7       1.00      1.00      1.00     18591
           8       1.00      1.00      1.00     47633
           9       1.00      1.00      1.00     16075
          10       1.00      1.00      1.00     29762

    accuracy                           0.91    251068
   macro avg       0.93      0.93      0.93    251068
weighted avg       0.91      0.91      0.91    251068

[[17345     9     3     0     0     0     0     2     

In [ ]:
def dt_classifier(data, device_name, scaling=False,):
    X = data.drop(['label'], axis=1)
    y = data['label']
    print(f'Original Shape:{X.shape}, {y.shape}')
    if scaling == False:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        model_name = f'{device_name}_without_scaling_unbalanced_model_decision_tree_gini.pkl'
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        model_name = f'{device_name}_with_scaling_unbalanced_model_decision_tree_gini.pkl'
    
    clf = DecisionTreeClassifier(criterion='gini', random_state= 1234, max_depth  = 5 )
    start=time.time()
    model_res = clf.fit(X_train, y_train)
    end=time.time()
    tt=end-start
    y_pred = model_res.predict(X_test)
    y_pred_prob = model_res.predict_proba(X_test)
    lr_probs = y_pred_prob[:,1]
    ac = accuracy_score(y_test, y_pred)
    importances = pd.DataFrame({'feature':X.columns,'importance':np.round(clf.feature_importances_,3)})
    importances = importances.sort_values('importance',ascending=False).set_index('feature')
    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    print('Decision tree')
    print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
    print(f"Training time: {tt}s")
    print('Accuracy=%.3f' % (ac))
    print('f1-score=%.3f' % (f1))
    print('Confusion Matrix = \n', cm)
    print('Classification Report',classification_report(y_test, y_pred))

In [ ]:
dt_classifier(temp_df,'device2',True)

Original Shape:(836891, 51), (836891,)
Decision tree
Recall = 0.7196297417432728
Training time: 11.728534698486328s
Accuracy=0.720
f1-score=0.656
Confusion Matrix = 
 [[17356     0     4     0     0     0     0     0     0     0     0]
 [ 8696     0     4     0     0     0     0     0     0     0     0]
 [ 8503     0     3     0     0     0     0     0     0     0     0]
 [    0     0     0    21 26693     0     0     0     0     0     6]
 [    0     0     1     0 31103     0     0     0     0     0    28]
 [10354     0     0     0     0  7019     0     0     0     0     0]
 [    0     0     3     0     0     0 29213     0     0     0     0]
 [    0     0     1     0     0     0     0 18590     0     0     0]
 [   17     0     0     1     0     0     0     2 47613     0     0]
 [16074     0     1     0     0     0     0     0     0     0     0]
 [    0     0     4     0     0     0     0     0     0     0 29758]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report               precision    recall  f1-score   support

           0       0.28      1.00      0.44     17360
           1       0.00      0.00      0.00      8700
           2       0.14      0.00      0.00      8506
           3       0.95      0.00      0.00     26720
           4       0.54      1.00      0.70     31132
           5       1.00      0.40      0.58     17373
           6       1.00      1.00      1.00     29216
           7       1.00      1.00      1.00     18591
           8       1.00      1.00      1.00     47633
           9       0.00      0.00      0.00     16075
          10       1.00      1.00      1.00     29762

    accuracy                           0.72    251068
   macro avg       0.63      0.58      0.52    251068
weighted avg       0.76      0.72      0.66    251068



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
